In [75]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
!pip install webcolors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
# Import libraries 
import zipfile
from shutil import copyfile
from google.colab import drive
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import keras
from PIL import *
import webcolors

In [78]:
# Connect to Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [79]:
copyfile('gdrive/MyDrive/ProgettoDeepLearning/Dataset.zip', 'Dataset.zip' ) 

'Dataset.zip'

In [80]:
# Extract zip file
zip = zipfile.ZipFile('Dataset.zip')
zip.extractall()
zip.close

<bound method ZipFile.close of <zipfile.ZipFile filename='Dataset.zip' mode='r'>>

In [81]:
# Save directories of images and associates masks
img_dir = '/content/Dataset/701_StillsRaw_full'
mask_dir = '/content/Dataset/LabeledApproved_full'

Define pairs between images and masks

In [82]:
def make_pair(img, mask):
    pairs_path = []
    pairs = []
    for im in img:
      for ms in mask:
        if im.split(".png")[0] == ms.split("_L.png")[0]:
          pairs_path.append((img_dir + '/' + im, mask_dir + '/' + ms))
          pairs.append((im ,((im.split('.png')[0]) +"_L.png")))
           
       
      
      
    
    return pairs_path


In [83]:
# Create file paths from file names
file_list = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
mask_list = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]

file_list = sorted(file_list)
mask_list = sorted(mask_list)

pair_list = make_pair(file_list, mask_list)

In [84]:
len(set(pair_list))

701

Load the trainig set:

In [85]:
train_df = pd.read_csv("/content/gdrive/MyDrive/ProgettoDeepLearning/DATA/train_df.csv", index_col = 0)

In [86]:
# Create a list of tuples with image paths and mask paths for training set
records = train_df.to_records(index=False)
train_data = list(records)

Create two dictionaries:


*   *color_pixel*: associate to each color (RGB code) the total number of pixel in the training set of its color;
*   *color_n_img*: associate to each color (RGB code) the number of images that presents at least one pixel of its color.



In [87]:
color_pixel = {}
color_n_img = {} 

for i in range(len(train_df)):
  with Image.open(train_df['train_mask'][i]) as image:
          color_count = {}
          width, height = image.size
          rgb_image = image.convert('RGB')
          # Iterate through each pixel in the image and keep a count per unique color
          for x in range(width):
              for y in range(height):
                  rgb = rgb_image.getpixel((x, y))

                  if rgb in color_count:
                      color_count[rgb] += 1
                  else:
                      color_count[rgb] = 1

          color_index = 1
          for color, count in color_count.items():
              
              # Inizialize new key "color"
              if color not in color_pixel.keys():
                color_pixel[color] = count
              else:
                color_pixel[color] += count # Increase the counting of pixels associated with the color
              
              # Initialize new key "color" 
              if color not in color_n_img.keys():
                color_n_img[color] = 0
              
              # If the mask presents at least one pixel of "color"
              if count != 0:
                color_n_img[color] += 1 # Increase the counting of images the persents the color

              color_index += 1

Print the results:

In [88]:
color_pixel

{(0, 0, 0): 7143415,
 (0, 0, 64): 9711,
 (0, 0, 192): 15281396,
 (0, 64, 64): 853398,
 (0, 128, 64): 31382,
 (0, 128, 192): 1199410,
 (64, 0, 128): 8317151,
 (64, 0, 192): 79556,
 (64, 64, 0): 1524364,
 (64, 64, 128): 3334748,
 (64, 128, 64): 12695,
 (64, 128, 192): 1735192,
 (64, 192, 0): 3447067,
 (64, 192, 128): 845460,
 (128, 0, 0): 54578790,
 (128, 0, 192): 4109940,
 (128, 64, 64): 933794,
 (128, 64, 128): 66145858,
 (128, 128, 0): 26073003,
 (128, 128, 64): 1622766,
 (128, 128, 128): 38279911,
 (128, 128, 192): 620338,
 (192, 0, 64): 32515,
 (192, 0, 128): 212581,
 (192, 0, 192): 40456,
 (192, 128, 64): 79113,
 (192, 128, 128): 290521,
 (192, 128, 192): 1288803,
 (192, 192, 0): 2012810,
 (192, 192, 128): 2475056}

In [89]:
# Counting the total number of pixels seen in the training set
count =0
for i in color_pixel.values():
  count += i

# The total number of pixel must be the number of images in the trainig set (351) x image height (960) x image width (720) = 242611200
count

242611200

In [90]:
color_n_img

{(0, 0, 0): 351,
 (0, 0, 64): 9,
 (0, 0, 192): 335,
 (0, 64, 64): 232,
 (0, 128, 64): 3,
 (0, 128, 192): 183,
 (64, 0, 128): 325,
 (64, 0, 192): 122,
 (64, 64, 0): 318,
 (64, 64, 128): 181,
 (64, 128, 64): 12,
 (64, 128, 192): 147,
 (64, 192, 0): 233,
 (64, 192, 128): 97,
 (128, 0, 0): 345,
 (128, 0, 192): 348,
 (128, 64, 64): 212,
 (128, 64, 128): 351,
 (128, 128, 0): 314,
 (128, 128, 64): 297,
 (128, 128, 128): 350,
 (128, 128, 192): 30,
 (192, 0, 64): 12,
 (192, 0, 128): 59,
 (192, 0, 192): 11,
 (192, 128, 64): 76,
 (192, 128, 128): 206,
 (192, 128, 192): 87,
 (192, 192, 0): 153,
 (192, 192, 128): 350}

Import matching between labels and RGB codes for checking the results:

In [91]:
copyfile('gdrive/MyDrive/ProgettoDeepLearning/label_colors.txt', 'label_colors.txt') 

'label_colors.txt'

In [92]:
label = pd.read_csv('label_colors.txt', header = None)

In [93]:
lines = []
for elemento in label[0]:
  lines.append(elemento.strip().split("\t"))

In [94]:
# Create code list and label list
code = []
names = []
for elemento in lines:
  if len(elemento) == 2:
    code.append(elemento[0])
    names.append(elemento[1])
  else:
    code.append(elemento[0])
    names.append(elemento[2])

In [95]:
# Create a label dataframe
label_color = pd.DataFrame(names, columns = ['Names'])

In [96]:
# Create a list of lists with [R, G, B] split codes 
class_map = []
for i in code: 
  R,G,B = i.split(' ')
  class_map.append(np.array([int(R), int(G), int(B)]))

In [97]:
# Add RGB code column 
label_color['Code'] = class_map

In [98]:
label_color

,Names,Code
0,Animal,"[64, 128, 64]"
1,Archway,"[192, 0, 128]"
2,Bicyclist,"[0, 128, 192]"
3,Bridge,"[0, 128, 64]"
4,Building,"[128, 0, 0]"
5,Car,"[64, 0, 128]"
6,CartLuggagePram,"[64, 0, 192]"
7,Child,"[192, 128, 64]"
8,Column_Pole,"[192, 192, 128]"
9,Fence,"[64, 64, 128]"
